# Fonctions Utiles 

## Variables Paramètres

In [1]:
KEY = "RGAPI-380dba88-f9aa-41b0-b833-1d72ab6d54f5"

In [2]:
import requests
import pandas as pd
import json
import time

## Mémoïzation ?
Affin d'éviter de demander plusieurs fois la même chose, on pourrait avoir une classe 'Joueur' qui permet de centraliser les infos, pour que toutes les méthodes y aient accès. Dans ce cas, les fonctions de calcul de données deviendraient des méthodes de cette classe.

In [3]:
class Player:
    pass

## Gestion des "bad requests"

In [48]:
def badRequestsHandler(url):
    """
    Permet d'analyser le code de retour de la requête pour gérer les problèmes éventuels.
    Pas sûr que cela permette de gérer tous les problèmes, certains seront probablement spécifiques.
    """
    r = requests.get(url)
    if r.status_code == 429:
        print("Quota dépassé ...", end = "\r")
        time.sleep(int(r.headers["Retry-After"])+2)
        r = requests.get(url)
    elif r.status_code == 400:
        raise Exception("Requête invalide")
    elif r.status_code == 401:
        raise Exception("Unauthorized")
    elif r.status_code == 403:
        raise Exception("Non autorisé: vérifiez la clé")
    elif r.status_code == 404:
        raise Exception("Données non trouvées")
    elif r.status_code == 405:
        raise Exception("Méthode non autorisée")
    elif r.status_code == 415:
        raise Exception("Unsupported media type")
    elif r.status_code == 500:
        raise Exception("Internal server error")
    elif r.status_code == 502:
        raise Exception("Bad gateway")
    elif r.status_code == 503:
        raise Exception("Service non disponible")
    elif r.status_code == 504:
        raise Exception("Gateway timeout")
    elif r.status_code == 200:
        return r.json()

## Récupération des données brutes d'un joueur

In [27]:
def requestSummonerInfo(summoner_name,key):
    """
    Renvoie un dictionnaire contenant des infos sur le joueur 'summoner_name', à savoir :
    accountId :    string - Encrypted account ID. Max length 56 characters.
    profileIconId:    int - ID of the summoner icon associated with the summoner.
    revisionDate:    long - Date summoner was last modified specified as epoch milliseconds. The following events will update this timestamp: summoner name change, summoner level change, or profile icon change.
    name:          string - Summoner name.
    id:            string - Encrypted summoner ID. Max length 63 characters.
    puuid:         string - Encrypted PUUID. Exact length of 78 characters.
    summonerLevel:   long - Summoner level associated with the summoner.
    """
    return badRequestsHandler(f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={key}")

requestSummonerInfo('Skeanz',KEY)

{'id': 'oIQG68UUDKSt7bUQlBtSBLqzKCelfv4jSrzooNJ_L4iosa5jrtahspGZUw',
 'accountId': 's4GxVhnvULshQkAN0icIxFhnOBuY3rcm_dolU43KsJLeAY9HN6RbMrg3',
 'puuid': 'TGabv96Z9DC3LHkHNJoqS4j-HJIjLUqqhoyeeJ4FmJwudqjIdh57uvPZbUQJvGd9xtr0ShdUilvzpw',
 'name': 'Skeanz',
 'profileIconId': 7,
 'revisionDate': 1666001193416,
 'summonerLevel': 302}

In [46]:
def requestMostRecentGamesId(puuid,key, nb_of_games=None, nb_of_days_ago=None):
    """
    Si nb_of_games    != None : Récupère les 'nb_of_games' dernières parties jouées par le joueur.
    Si nb_of_days_ago != None : Récupère toutes les parties jouées par le joueur dans les 'nb_of_days_ago' jours.
    Si les deux sont != None ou les deux sont == None, renvoie une erreur.
    Renvoie une liste contenant les id de ces parties.
    """
    if (nb_of_games==None and nb_of_days_ago==None) or (nb_of_games != None and nb_of_days_ago != None):
        raise Exception("Erreur d'inputs, les paramètres nb_of_games et nb_of_days_ago ne peuvent pas tous les deux avoir la même valeur")
    elif nb_of_games != None:
        res_games = []
        start_index = 0 # très important, sinon on récupère en boucle les 100 mêmes games
        while nb_of_games >100:    #la valeur max du "count" pour avoir une liste de match est de 100
            r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start_index}&count=100&api_key={key}")
            if r == []: # on a atteint le max de games du joueur
                break
            start_index+=100
            res_games += r
            nb_of_games-=100
            
        r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start_index}&count={nb_of_games}&api_key={key}")
        res_games += r

    elif nb_of_days_ago != None :
        res_games = []
        start_index = 0
        startTime = int(time.time()-24*60*60*nb_of_days_ago) # l'horaire de départ est "aujourd'hui"- nb_of_days_ago si on veut les games dans les nb_of_days_ago derniers jours
        while True: 
            r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={startTime}&start={start_index}&count=100&api_key={key}")
            if r == [] : #quand il n'y a plus de games à récup, on sort de la boucle
                break
            start_index+=100
            res_games += r
    return res_games

# GamesId = requestMostRecentGamesId('TGabv96Z9DC3LHkHNJoqS4j-HJIjLUqqhoyeeJ4FmJwudqjIdh57uvPZbUQJvGd9xtr0ShdUilvzpw',KEY,nb_of_games=300)
# GamesId_7_last_days = requestMostRecentGamesId('TGabv96Z9DC3LHkHNJoqS4j-HJIjLUqqhoyeeJ4FmJwudqjIdh57uvPZbUQJvGd9xtr0ShdUilvzpw',KEY,nb_of_days_ago=7)
# print(GamesId_7_last_days)

In [29]:
def requestPlayersOfARank(queue,tier,division,number_of_players,key):
    """ 
    Pour récupérer des joueurs de même ELO

    Args
    ----
    queue : String
        Quelle queue ? Parmis RANKED_SOLO_5x5,RANKED_FLEX_SR,RANKED_FLEX_TT
    tier : String
        Nom de la ligue en anglais (ex : GOLD, PLATINIUM...)
    division : String
        numéro de la division : I,II,III,IV
    number_of_players : Int
        Nombre de joueurs voulus
    
    Returns
    -------
    Dict
        Dictionnaire rassemblant les informations des différents joueurs
        Attention : le nombre de joueurs sera arrondi aux 205 supérieurs (par exemple pour 320, 410 seront donnés)
    """

    tier=tier.upper()
    if number_of_players > 205 : # les pages renvoyées par chaque requête contiennent 205 joueurs
        liste_joueurs = []
        for i in range(1,(number_of_players//205)+2):
            print(i)
            r = badRequestsHandler(f"https://euw1.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}I?page={i}&api_key={key}")
            if r == []:
                return liste_joueurs # on a atteint le nombre de joueur total de la division
            liste_joueurs += r
    else :
        liste_joueurs = badRequestsHandler(f"https://euw1.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}I?page=1&api_key={key}")
    return liste_joueurs

# liste = requestPlayersOfARank("RANKED_SOLO_5x5","DIAMOND","I",100000,KEY)
# print(len(liste))

In [40]:
def requestRankedInfo(summoner_id,key):
    """
    Renvoie un dictionnaire de dictionnaire contenant des infos sur les résultats du joueur en partie classées, à savoir :
    FLEXQ: {
        tier: string - Iron -> Challenger
        rank: string - IV -> I
        lp: int - league points
        wins: int
        losses: int
    }
    SOLOQ: {pareil}
    """
    return (badRequestsHandler(f"https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/{summoner_id}?api_key={key}"))

# print(requestRankedInfo('oIQG68UUDKSt7bUQlBtSBLqzKCelfv4jSrzooNJ_L4iosa5jrtahspGZUw',KEY))

## Algorithme général pour la création du Dataset

In [51]:

def infos_games(queue,tier,division,number_of_players,number_of_games,key):
    id_games = []
    liste_joueurs = requestPlayersOfARank("RANKED_SOLO_5x5","DIAMOND","I",number_of_players,key) # chaque élément de la liste contient le nom des joueurs
    liste_joueurs_finale = liste_joueurs[:number_of_players]
    i=0
    while len(id_games) <number_of_games:
        info_joueur = requestSummonerInfo(liste_joueurs_finale[i]['summonerName'],key)
        parties_joueur = requestMostRecentGamesId(info_joueur['puuid'],key, nb_of_games=1000) #recup de toute les parties du joueur (on considère qu'il en a fait - de 100 000)
        id_games+=parties_joueur
        i+=1

    # Une fois les games récup, on supprime les doublons car ils sont inutiles et utilise des requêtes
    id_games_set = set(id_games)
    id_games_uniques = (list(id_games_set))
    if len(id_games_uniques) > number_of_games : # on fait attention de ne pas récup trop de games
        id_games_finale = id_games_uniques[:number_of_games]
    else : 
        id_games_finale = id_games_uniques

    dataset_games = []
    for game in id_games_finale:
        infos_game = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/{game}?api_key={key}")
        dataset_games +=[infos_game]
    
    return(dataset_games)

print(len(infos_games("RANKED_SOLO_5x5","DIAMOND","I",10,5,KEY)))

[{'metadata': {'dataVersion': '2', 'matchId': 'EUW1_6088272996', 'participants': ['WYGmyE6ZnCmtIaEHmPUdfpnWMZBcYUJ27mLrvlTx5kGUOpcD0O-FPalLTqmTxCFo00Av7JHix_q9Wg', '0bzS3IKVGCfiCuxEqhB4nZ-E6sl7c8QV4-xUMtQtCeTfK_EwQuP8rQHqbQb_b1BzgbO481sXDg0J-g', '5FmlfDO7hEZ7UfNLctkeMll8SNjiXTAGo2W6GUPZRDxQwg_dgsRC5Ewppl8S8X1-wSoop03ZO9DLFw', 'x4XGg6vAV4P9AgSkpk5PxwDQi8K4gg_uTYhaFudo7eYb6CU-SaiKv1Bx5UtCeNRJVvjEkDadCc8wUw', 'w3IAOS3sqkplsSxSXr_T0pC_y3lQ55HOCpO0ai6NliFZvLlHWDQWdcxDhqBiCiRsrHxfPCCwom9VxQ', '6qjJBLsAmd3TMcW2hZcUaqjXRLwjWD1yYZuzPb0xctB_E40CIF6ZREzuLQnDV4q5_d9yB1QyHoc-7Q', '_hGSQPckekJec_UzSwB1c_waDZ134YeaiO0SeBCNfW1T90fWoCA9ZDZ52Vqhnr2AcklMdohFfI821A', 'O-dYLnCYIdkxb5OC_JOIgTYJKorSYQRaos2TK584jZgdgb5Fk-XAV5ifGtMLrC5ul7QIVjD3wHGEvA', 'nvnVdbEQRDOGwJf-9AtvwrjtaCIYpkHUdImXaVA-9RAYtsq20JKvO-rzXWZFQR8GWFhCIkDMEOvROQ', 'vKidGnlLqbSgxkfz_nj_KxcCDQqWyGXh-x9LwpRCQ6jKBOGkcnw6fNCUi0Ub_fLw23OiCINsd-6u1w']}, 'info': {'gameCreation': 1664651628708, 'gameDuration': 1605, 'gameEndTimestamp': 1664653258002,

## Calculs de données

In [ ]:
def getWinrate(nb_of_games=0):
    """
    Calcule le winrate (float) et le KDA moyen (float,float,float) sur les 'nb_of_games' dernières parties.
    Si nb_of_games = 0, alors on fait le calcul sur toute la saison.
    Renvoie un dictionnaire de la forme:
    {
        wins: int
        losses: int
        winrate: float
        meanKDA: (float,float,float)
    }
    """
    pass

In [ ]:
def getWinrateOnChampions(nb_of_games=0):
    """
    Calcule le winrate et le KDA moyen sur chaque champion sur les 'nb_of_games' dernières parties
    Si nb_of_games = 0, alors on fait le calcul sur toute la saison.
    Renvoie un dictionnaire de la forme :
    {
        Aatrox: {
            wins: int
            losses: int
            winrate: float
            meanKDA: (float,float,float)
        }
        ...
    }
    """
    pass